In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
import re

In [ ]:
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# **Text Preprocessing**

In [ ]:
df=df = pd.read_csv("/kaggle/input/turkcehaber/data.csv")
df.head()

In [ ]:
df.rename(columns={'Haber Gövdesi': 'haber', 'Sınıf': 'sinif'}, inplace=True)

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.haber.duplicated(keep="first").value_counts()

In [ ]:
df.drop_duplicates(subset="haber",keep="first",inplace=True,ignore_index=True)

In [ ]:
df.describe()

In [ ]:
df.sinif.unique()

In [ ]:
news_dict={'Magazin':0,'Siyaset':1,'Sağlık':2,'Spor':3,'Kültür-Sanat':4,'Turizm':5,'Finans-Ekonomi':6,'Bilim-Teknoloji':7,'Çevre':8}
df["sinif"]=df["sinif"].map(news_dict)

In [ ]:
df.isnull().any()

In [ ]:
def clean_text(text):
    unwanted_pattern = r'[!.\n,\':“”,?@#x*-+/%&"\'-]'
    regex = re.compile(unwanted_pattern)
    cleaned_text = regex.sub(" ", text)
    return cleaned_text

In [ ]:
df["haber"] = df["haber"].astype(str).apply(clean_text)
df["haber"] = df["haber"].str.replace("[\d]", "")
df["haber"] = df["haber"].str.lower()
df

# **Analyze Texts**

In [ ]:
df['words'] = [len(x.split()) for x in df['haber'].tolist()]

In [ ]:
df

In [ ]:
df['words'].describe()

df.groupby(['sinif'])['words'].describe()

In [ ]:
min_word_size = 50
max_word_size= 1200
df[df['words']<min_word_size].count()
df[df['words']>max_word_size].count()

In [ ]:
df= df[df['words']>=min_word_size]
df= df[df['words']<=max_word_size]

In [ ]:
df

In [ ]:
word_freq= df.haber.str.split(expand=True).stack().value_counts()
word_freq=word_freq.reset_index(name='freq').rename(columns={'index': 'word'})

In [ ]:
top_50_frequent_words = word_freq[:50]
top_50_frequent_words